# `CrystalMap` API

* Quick and dirty description of the (more or less) full `CrystalMap` API so far.
* Will of course remove lots of stuff here and make it user friendly before merging into orix-demos.
* Notable things missing:
    * `__setitem__`
* Many try/except and input checks are implemented. Try to break things. An explanatory error message *should* be raised.

<!--
# Introduction

This notebook illustrates clustering of Ti crystal orientations using data obtained from a highly deformed specimen, using EBSD.

This functionaility has been checked to run in orix-0.2.0 (December 2019). Bugs are always possible, do not trust the code blindly, and if you experience any issues please report them here: https://github.com/pyxem/orix-demos/issues
-->

# Contents

1. <a href='#load'> Temporary load functions</a>
2. <a href='#init'> Initialize a CrystalMap object</a>
3. <a href='#crystalmap'> Inspect a CrystalMap object</a>
4. <a href='#phaselist'> PhaseList</a>
5. <a href='#phase'> Phase</a>
6. <a href='#examples'> Examples</a>

Import orix classes and various dependencies

In [1]:
%matplotlib qt5

# Important external dependencies
import os
import re
import numpy as np
import matplotlib.pyplot as plt
import h5py  # Needed for h5ebsd reader

# orix dependencies (not tested)
from orix.crystal_map import CrystalMap, PhaseList, Phase

# <a id='load'></a> 1. Temporary load functions

### .ang file created by EMsoft's EMdpmerge program

Not at all done, just something quick to read all necessary bits.

In [2]:
def load_ang(filename):
    data = np.loadtxt(filename)
    rows, cols = data.shape

    euler = np.radians(data[:, :3])
    x = data[:, 3]
    y = data[:, 4]
    iq = data[:, 5]
    ci = data[:, 6]

    fit = np.ones_like(iq)
    if cols > 8:
        fit = data[:, 7]
        phase = data[:, 8]
    else:
        phase = data[:, 7]

    properties = {
        'iq': iq,
        'ci': ci,
        'fit': fit,
    }

    # Read header
    header = []
    for line in open(filename):
        l = line.strip()
        if l.startswith("#"):
            header.append(l.rstrip())

    def get_phase_from_ang(header):
        phases = {}
        for i, l in enumerate(header):
            if l.startswith("# Phase"):
                phase_id = re.search("# Phase( +)([0-9]+)", l).group(2)
                material_name = ''
                formula = ''
                try:
                    material_name = re.search(
                        "# MaterialName( +)([A-z]+)/([A-z]+)", header[i + 1],
                    ).group(2)
                    formula= re.search(
                        "# Formula( +)([A-z]+)/([A-z]+)", header[i + 2]).group(
                        2)
                except (AttributeError, IndexError):
                    material_name = re.search(
                        "# MaterialName( +)([A-z]+)", header[i + 1],
                    ).group(2)
                    formula= re.search(
                        "# Formula( +)([A-z]+)", header[i + 2]).group(2)
                phases[phase_id] = {
                    'material_name': material_name,
                    'formula': formula,
                    'symmetry': re.search(
                        "# Symmetry( +)([A-z0-9]+)", header[i + 4],
                    ).group(2),
                }
        return phases

    # Get phases
    phases = get_phase_from_ang(header)

    # Get symmetries and minerals
    symmetry = []
    mineral = []
    for p in phases.values():
        symmetry.append(p['symmetry'])
        mineral.append(p['material_name'])

    return x, y, phase, euler, properties, symmetry, mineral

### .h5 file created by EMsoft's EMEBSDDI program

Not at all done, just something quick to read all necessary bits.

In [3]:
def load_h5ebsd(filename, refined=False, **kwargs):
    mode = kwargs.pop("mode", "r")
    f = h5py.File(filename, mode=mode, **kwargs)

    scan_group = f["Scan 1"]
    ebsd_group = scan_group["EBSD"]
    data_group = ebsd_group["Data"]
    header_group = ebsd_group["Header"]

    # Get scan coordinates
    nx = header_group["nColumns"][()]
    ny = header_group["nRows"][()]
    dx = header_group["Step X"][()]
    dy = header_group["Step Y"][()]
    x = np.tile(np.arange(nx*dx, step=dx), reps=ny)
    y = np.tile(np.arange(ny*dy, step=dy), reps=nx)
    z = np.zeros_like(x)

    # Get phase
    phase = data_group["Phase"][()]

    # Get crystal symmetry
    point_group = re.search(
        r"\[([A-Za-z0-9_]+)\]",
        ebsd_group["Header/Phase/1/Point Group"][()][0].decode()
    ).group(1)

    # Get orientations
    if refined:
        orientation_dataset_name = "RefinedEulerAngles"
    else:
        orientation_dataset_name = "EulerAngles"
    euler = data_group[orientation_dataset_name][()]

    # Get properties
    properties = {}
    quality_metric_names = [
        "AvDotProductMap",
        "CI",
        "IQ",
        "ISM",
        "OSM",
    ]
    for metric_name in quality_metric_names:
        metric = data_group[metric_name][()]
        if metric.ndim > 1:
            metric = metric.ravel()
        properties[metric_name] = metric

    return x, y, z, phase, euler, properties, point_group

### Utility program to get map shape and step size from x, y arrays

All this will be done internally...

In [4]:
def get_shape_and_step_sizes(x, y=None, z=None):
    shape = []
    step_sizes = []
    for i, direction in enumerate([z, y, x]):
        if direction is not None:
            unique_sorted = np.sort(np.unique(direction))
            step = unique_sorted[1] - unique_sorted[0]
            length = int((direction.max() + step) / step)
            shape.append(length)
            step_sizes.append(step)
    return tuple(shape), step_sizes

# <a id='init'></a> 2. Initialize a CrystalMap object

In [5]:
datadir = '/home/hakon/phd/data/jarle_emsoft/sdss/emsoft/'
fname = 'sdss_ferrite_austenite.ang'
file = os.path.join(datadir, fname)

x, y, phase, euler, props, symmetry, mineral = load_ang(file)

Reshape 1D arrays to map shape (will be done internally when current readers are expanded)

In [6]:
shape, step_sizes = get_shape_and_step_sizes(x, y)
euler = euler.reshape(shape + (3,))
phase = phase.reshape(shape)
prop = {}
for k, v in props.items():
    prop[k] = v.reshape(shape)

Print `__init__` docstring

In [7]:
print(CrystalMap.__init__.__doc__)    


        Parameters
        ----------
        rotations : numpy.ndarray, optional
            Rotation of each pixel.
        phase_id : numpy.ndarray, optional
            Phase ID of each pixel.
        phase_name : str or list of str, optional
            Name of phases.
        symmetry : str or list of str, optional
            Point group of crystal symmetries of phases in the map.
        prop : dict of numpy.ndarray, optional
            Dictionary of quality metrics or other properties of each
            pixel.
        indexed : numpy.ndarray
            Boolean array with True for indexed pixels.
        dx : numpy.ndarray, optional
            Step sizes in each map direction.
        


Create crystal map

In [8]:
cm = CrystalMap(
    rotations=euler,
    phase_id=phase,
    phase_name=mineral,
    symmetry=symmetry,
    prop=prop,
    dx=step_sizes,
)

# <a id='crystalmap'></a> 3. CrystalMap

Print class description

In [9]:
print(CrystalMap.__doc__)

Crystallographic map storing rotations, phases and pixel properties.

    Phases and pixel properties are stored in the map shape, while
    rotations are always stored as a 1D array.

    This class is inspired by the EBSD class available in MTEX
    [Bachmann2010]_.

    Attributes
    ----------
    all_indexed : bool
        Whether all pixels are indexed.
    dx : numpy.ndarray
        Step sizes in each map direction.
    indexed : numpy.ndarray
        Boolean array with True for indexed pixels.
    ndim : int
        Number of map dimensions.
    orientations : orix.quaternion.orientation.Orientation
        Orientations of each pixel. Always 1D.
    phase_id : numpy.ndarray
        Phase ID of each pixel as imported.
    phases : orix.crystal_map.PhaseList
        List of phases with their IDs, names, crystal symmetries and
        colors (possibly more than are in the map).
    phases_in_map : orix.crystal_map.PhaseList
        List of phases in the map, with their IDs, names

`__repr__` (inspired by MTEX)

In [10]:
cm

Phase  Orientations   Name       Symmetry  Color     
1      5657 (48.4%)   austenite  432       tab:blue  
2      6043 (51.6%)   ferrite    432       tab:orange
Properties: iq, ci, fit
Scan unit: um

Custom, private attributes

In [11]:
[i for i in dir(cm) if i.startswith('_') and not i.endswith('__')]

['_dx',
 '_indexed',
 '_phase_id',
 '_phases',
 '_prop',
 '_rotations',
 '_scan_unit']

Public attributes and methods

In [12]:
[i for i in dir(cm) if not i.startswith('_')]

['all_indexed',
 'deepcopy',
 'dx',
 'indexed',
 'ndim',
 'orientations',
 'phase_id',
 'phases',
 'phases_in_map',
 'pixel_id',
 'plot_phase',
 'plot_prop',
 'prop',
 'rotations',
 'scan_unit',
 'shape',
 'size']

Docstrings of public attributes and their values in the current CrystalMap instance

In [13]:
print(CrystalMap.all_indexed.__doc__)
cm.all_indexed

Return whether all map pixels are indexed.


True

In [14]:
print(CrystalMap.dx.__doc__)
cm.dx

Return pixel step size in each direction in scan units.


[1.5, 1.5]

In [15]:
print(CrystalMap.indexed.__doc__)
cm.indexed

Return boolean numpy.ndarray with indexed pixels set to True.


array([[ True,  True,  True, ...,  True,  True,  True],
       [ True,  True,  True, ...,  True,  True,  True],
       [ True,  True,  True, ...,  True,  True,  True],
       ...,
       [ True,  True,  True, ...,  True,  True,  True],
       [ True,  True,  True, ...,  True,  True,  True],
       [ True,  True,  True, ...,  True,  True,  True]])

In [16]:
print(CrystalMap.ndim.__doc__)
cm.ndim

Return number of map dimensions.


2

In [17]:
print(CrystalMap.orientations.__doc__)
cm['austenite'].orientations

Return an Orientation object, which is always 1D.


Orientation (5657,) 1, 432
[[-0.9982 -0.0082  0.0001 -0.0589]
 [-0.9982 -0.0082  0.0001 -0.0589]
 [-0.9982 -0.0082  0.0001 -0.0588]
 ...
 [-0.9982 -0.0014 -0.     -0.0593]
 [-0.9987 -0.0074  0.0001 -0.0506]
 [-0.9987 -0.0074  0.0001 -0.0506]]

In [18]:
print(CrystalMap.phase_id.__doc__)
cm.phase_id

Return numpy.ndarray of the phase ID of each map pixel.


array([[2, 1, 1, ..., 1, 1, 1],
       [2, 2, 1, ..., 1, 1, 1],
       [2, 2, 2, ..., 1, 1, 1],
       ...,
       [2, 2, 2, ..., 2, 2, 1],
       [2, 2, 2, ..., 2, 2, 1],
       [2, 2, 2, ..., 2, 1, 1]])

In [19]:
print(CrystalMap.phases.__doc__)
cm.phases  # similar CrystalMap's __repr__

Return a list of phases in the map (and potentially more).


Id  Name      Symmetry Color     
1   austenite 432      tab:blue  
2   ferrite   432      tab:orange

In [20]:
print(CrystalMap.phases_in_map.__doc__)
cm.phases_in_map

Return a list of phases in the map.


Id  Name      Symmetry Color     
1   austenite 432      tab:blue  
2   ferrite   432      tab:orange

In [21]:
print(CrystalMap.pixel_id.__doc__)
cm.pixel_id

Return map pixel IDs as a numpy.ndarray of same shape as map.


array([[    0,     1,     2, ...,   114,   115,   116],
       [  117,   118,   119, ...,   231,   232,   233],
       [  234,   235,   236, ...,   348,   349,   350],
       ...,
       [11349, 11350, 11351, ..., 11463, 11464, 11465],
       [11466, 11467, 11468, ..., 11580, 11581, 11582],
       [11583, 11584, 11585, ..., 11697, 11698, 11699]])

In [22]:
print(CrystalMap.prop.__doc__)
cm.prop

Return a dict of properties of each pixel.


{'iq': array([[24.4, 24. , 30.3, ..., 31.4, 29.1, 30.3],
        [23.1, 27. , 19.8, ..., 31.5, 32.7, 30.2],
        [27.5, 33.7, 24.3, ..., 31.5, 30.9, 30.1],
        ...,
        [22.4, 31.1, 32.1, ..., 35.8, 33.4, 25.1],
        [21.9, 35.5, 33.8, ..., 33. , 23.9, 23.5],
        [24.7, 33.2, 33.3, ..., 25.3, 25.8, 31.7]]),
 'ci': array([[0.799, 0.797, 0.825, ..., 0.826, 0.826, 0.828],
        [0.799, 0.815, 0.807, ..., 0.827, 0.828, 0.824],
        [0.797, 0.828, 0.809, ..., 0.829, 0.831, 0.827],
        ...,
        [0.798, 0.825, 0.831, ..., 0.833, 0.829, 0.8  ],
        [0.803, 0.83 , 0.835, ..., 0.828, 0.811, 0.818],
        [0.806, 0.831, 0.832, ..., 0.817, 0.809, 0.828]]),
 'fit': array([[1., 1., 1., ..., 1., 1., 1.],
        [1., 1., 1., ..., 1., 1., 1.],
        [1., 1., 1., ..., 1., 1., 1.],
        ...,
        [1., 1., 1., ..., 1., 1., 1.],
        [1., 1., 1., ..., 1., 1., 1.],
        [1., 1., 1., ..., 1., 1., 1.]])}

In [23]:
print(CrystalMap.rotations.__doc__)
cm.rotations

Return a Rotation object, which is always 1D.

        Must always be 1D because of possible masked elements in indexed
        attribute.
        


Rotation (11700,)
[[-0.9984 -0.0059 -0.     -0.057 ]
 [-0.9982 -0.0082  0.0001 -0.0589]
 [-0.9982 -0.0082  0.0001 -0.0589]
 ...
 [-0.9987 -0.006   0.0003 -0.0505]
 [-0.9987 -0.0074  0.0001 -0.0506]
 [-0.9987 -0.0074  0.0001 -0.0506]]

In [24]:
print(CrystalMap.scan_unit.__doc__)
cm.scan_unit

Return scan unit as a string.


'um'

In [25]:
print(CrystalMap.shape.__doc__)
cm.shape

Return shape of map in pixels.


(100, 117)

In [26]:
print(CrystalMap.size.__doc__)
cm.size

Return total number of map pixels.


11700

Docstrings of CrystalMap methods

In [27]:
print(CrystalMap.plot_phase.__doc__)

Return and plot map phases.

        Parameters
        ----------
        overlay : str, optional
            Map property to use as alpha value. The property is adjusted
            for maximum contrast.
        legend : bool, optional
            Whether to display a legend with phases in the map (default is
            ``True``).
        scalebar : bool, optional
            Whether to add a scalebar (default is ``True``) along the last
            map dimension.
        padding : bool, optional
            Whether to show white padding (default is ``True``). Setting
            this to false removes all white padding outside of plot,
            including pixel coordinate ticks.
        **kwargs :
            Optional keyword arguments passed to
            :meth:`matplotlib.pyplot.imshow`.

        Returns
        -------
        phase : numpy.ndarray
            Phase array as passed to :meth:`matplotlib.pyplot.imshow` with
            colors and potential alpha value if a valid

In [28]:
print(CrystalMap.plot_prop.__doc__)

Plot of a map property.

        Parameters
        ----------
        prop : str
            The property in `prop` to plot.
        colorbar : bool, optional
            Whether to add a colorbar (default is ``True``).
        scalebar : bool, optional
            Whether to add a scalebar (default is ``True``) along the last
            map dimension.
        padding : bool, optional
            Whether to show white padding (default is ``True``). Setting
            this to ``False`` removes all white padding outside of plot,
            including pixel coordinate ticks.
        **kwargs :
            Optional keyword arguments passed to
            :meth:`matplotlib.pyplot.imshow`.

        Returns
        -------
        fig : matplotlib.figure.Figure
            Top level container for all plot elements.
        ax : matplotlib.axes.Axes
            Axes object returned by :meth:`matplotlib.pyplot.subplots`.
        im : matplotlib.image.AxesImage
            Image object return

Indexing/slicing ...

... by map position (slices)

In [29]:
(y0, y1) = (20, 40)
(x0, x1) = (50, 60)
cm1 = cm[y0:y1, x0:x1]
cm1

Phase  Orientations   Name       Symmetry  Color     
1      148 (74.0%)    austenite  432       tab:blue  
2      52 (26.0%)     ferrite    432       tab:orange
Properties: iq, ci, fit
Scan unit: um

In [30]:
cm1.size

200

... by phase name

In [31]:
cm2 = cm['austenite']
cm2

Phase  Orientations   Name       Symmetry  Color   
1      5657 (100.0%)  austenite  432       tab:blue
Properties: iq, ci, fit
Scan unit: um

In [32]:
# np.sum(cm2.indexed) / cm.size
cm2.size / cm.size

0.4835042735042735

In [33]:
cm3 = cm['austenite', 'ferrite']
cm3

Phase  Orientations   Name       Symmetry  Color     
1      5657 (48.4%)   austenite  432       tab:blue  
2      6043 (51.6%)   ferrite    432       tab:orange
Properties: iq, ci, fit
Scan unit: um

... by (chained) conditional(s)

In [34]:
cm4 = cm[cm.phase_id == 1]
cm4

Phase  Orientations   Name       Symmetry  Color   
1      5657 (100.0%)  austenite  432       tab:blue
Properties: iq, ci, fit
Scan unit: um

In [35]:
cm5 = cm[cm.ci > 0.81]
cm5

Phase  Orientations   Name       Symmetry  Color     
1      4092 (44.8%)   austenite  432       tab:blue  
2      5035 (55.2%)   ferrite    432       tab:orange
Properties: iq, ci, fit
Scan unit: um

In [36]:
cm6 = cm[(cm.iq > np.mean(cm.iq)) & (cm.phase_id == 1)]
cm6

Phase  Orientations   Name       Symmetry  Color   
1      1890 (100.0%)  austenite  432       tab:blue
Properties: iq, ci, fit
Scan unit: um

Add property

In [37]:
cm.prop['ci_times_iq'] = cm.ci * cm.iq
print(cm)
print("\n", cm.ci_times_iq)

Phase  Orientations   Name       Symmetry  Color     
1      5657 (48.4%)   austenite  432       tab:blue  
2      6043 (51.6%)   ferrite    432       tab:orange
Properties: iq, ci, fit, ci_times_iq
Scan unit: um

 [[19.4956 19.128  24.9975 ... 25.9364 24.0366 25.0884]
 [18.4569 22.005  15.9786 ... 26.0505 27.0756 24.8848]
 [21.9175 27.9036 19.6587 ... 26.1135 25.6779 24.8927]
 ...
 [17.8752 25.6575 26.6751 ... 29.8214 27.6886 20.08  ]
 [17.5857 29.465  28.223  ... 27.324  19.3829 19.223 ]
 [19.9082 27.5892 27.7056 ... 20.6701 20.8722 26.2476]]


# <a id='phaselist'></a> 4. PhaseList

In [38]:
print(PhaseList.__doc__)

A dictionary of phases in a crystallographic map.

    Each phase in the dictionary must has a unique phase id as key.

    Attributes
    ----------
    size : int
        Number of phases in list.
    names : list of str
        List of phase names.
    symmetries : list of orix.quaternion.symmetry.Symmetry
        List of phase crystal symmetries.
    colors : list of tuple
        List of tuples with three entries, RGB, defining phase colors.
    phase_ids : list of int
        List of unique phase indices in a crystallographic map as imported.
    


Get from CrystalMap

In [39]:
phases = cm.phases  # As shown above
phases

Id  Name      Symmetry Color     
1   austenite 432      tab:blue  
2   ferrite   432      tab:orange

Custom, private attributes

In [40]:
[i for i in dir(phases) if i.startswith('_') and not i.endswith('__')]

['_dict']

Public attributes

In [41]:
[i for i in dir(phases) if not i.startswith('_')]

['colors',
 'colors_rgb',
 'deepcopy',
 'names',
 'phase_ids',
 'size',
 'symmetries']

Inspect properties

In [42]:
print(PhaseList.size.__doc__)
phases.size

Number of phases in the list.


2

In [43]:
print(PhaseList.phase_ids.__doc__)
phases.phase_ids

Unique phase IDs in the list of phases.


[1, 2]

In [44]:
print(PhaseList.names.__doc__)
phases.names

List of phase names in the list.


['austenite', 'ferrite']

In [45]:
print(PhaseList.symmetries.__doc__)
phases.symmetries

List of crystal symmetries of phases in the list.


[Symmetry (24,) 432
 [[ 1.      0.      0.      0.    ]
  [ 0.7071  0.      0.      0.7071]
  [ 0.      0.      0.      1.    ]
  [-0.7071  0.      0.      0.7071]
  [ 0.5     0.5     0.5     0.5   ]
  [ 0.      0.      0.7071  0.7071]
  [-0.5    -0.5     0.5     0.5   ]
  [-0.7071 -0.7071  0.      0.    ]
  [ 0.      1.      0.      0.    ]
  [ 0.      0.7071  0.7071  0.    ]
  [ 0.      0.      1.      0.    ]
  [ 0.     -0.7071  0.7071  0.    ]
  [-0.5     0.5     0.5    -0.5   ]
  [ 0.      0.      0.7071 -0.7071]
  [ 0.5    -0.5     0.5    -0.5   ]
  [ 0.7071 -0.7071  0.      0.    ]
  [ 0.      0.7071  0.      0.7071]
  [-0.5     0.5     0.5     0.5   ]
  [-0.7071  0.      0.7071  0.    ]
  [-0.5    -0.5     0.5    -0.5   ]
  [ 0.      0.7071  0.     -0.7071]
  [ 0.5     0.5     0.5    -0.5   ]
  [ 0.7071  0.      0.7071  0.    ]
  [ 0.5    -0.5     0.5     0.5   ]],
 Symmetry (24,) 432
 [[ 1.      0.      0.      0.    ]
  [ 0.7071  0.      0.      0.7071]
  [ 0.      0.      0.

In [46]:
# Should perhaps add this property
#phases.symmetry_names

In [47]:
print(PhaseList.colors.__doc__)
phases.colors

List of phase color names in the list.


['tab:blue', 'tab:orange']

In [48]:
print(PhaseList.colors_rgb.__doc__)
phases.colors_rgb

List of phase color RGB values in the list.


[(0.12156862745098039, 0.4666666666666667, 0.7058823529411765),
 (1.0, 0.4980392156862745, 0.054901960784313725)]

Initialize a PhaseList...

In [49]:
print(PhaseList.__init__.__doc__)

None


... from input

In [50]:
PhaseList(
    names=['al', 'cu'],
    symmetries=['m-3m', 'm3m'],  # Note that m3m = m-3m
    colors=['lime', 'xkcd:violet'],
    phase_ids=[0, 1],
)

Id  Name  Symmetry Color      
0   al    m-3m     lime       
1   cu    m-3m     xkcd:violet

... from a list of Phase objects

In [74]:
al = Phase(name='al', symmetry='m-3m')
cu = Phase(name='cu')

#PhaseList([al, cu])  # Not working, will implement
PhaseList(al)

Id  Name  Symmetry Color   
0   al    m-3m     tab:blue

Index PhaseList...

In [52]:
print(PhaseList.__getitem__.__doc__)
# This doc should perhaps be moved to a more accessible place

Return a PhaseList or a Phase object, depending on input.

        Examples
        --------
        A PhaseList object can be indexed in multiple ways.
        >>> pl = PhaseList(names=['a', 'b'], symmetries=['1', '3'])
        >>> pl
        Id  Name  Symmetry  Color
        0   a     1         tab:blue
        1   b     3         tab:orange

        Return a Phase object

        >>> pl[0]  # Index with a single phase id
        <name: a. symmetry: 1. color: tab:blue>
        >>> pl['b']  # Index with a phase name
        <name: b. symmetry: 3. color: tab:orange>

        Return a PhaseList object

        >>> pl[0:]  # Index with slices
        Id  Name  Symmetry  Color
        0   a     1         tab:blue
        1   b     3         tab:orange
        >>> pl['a', 'b']  # Index with a tuple of phase names
        Id  Name  Symmetry  Color
        0   a     1         tab:blue
        1   b     3         tab:orange
        >>> pl[0, 1]  # Index with a tuple of phase ids
        Id  N

... by phase name

In [53]:
phases['austenite']

<name: austenite. symmetry: 432. color: tab:blue>

In [54]:
phases['austenite', 'ferrite']

Id  Name      Symmetry Color     
1   austenite 432      tab:blue  
2   ferrite   432      tab:orange

... by phase id

In [55]:
phases[1]

<name: austenite. symmetry: 432. color: tab:blue>

In [56]:
phases[1, 2]

Id  Name      Symmetry Color     
1   austenite 432      tab:blue  
2   ferrite   432      tab:orange

... by slice

In [57]:
phases[1:]

Id  Name      Symmetry Color     
1   austenite 432      tab:blue  
2   ferrite   432      tab:orange

Add phase to PhaseList

In [58]:
phases2 = phases.deepcopy()
phases2['sigma'] = '4/mmm'
phases2

Id  Name      Symmetry Color     
1   austenite 432      tab:blue  
2   ferrite   432      tab:orange
3   sigma     4/mmm    tab:green 

# <a id='phase'></a> 5. Phase

In [59]:
print(Phase.__doc__)

Name, crystal symmetry, and color of a phase in a crystallographic
    map.

    Attributes
    ----------
    name : str, None
        Phase name.
    symmetry : orix.quaternion.symmetries.Symmetry, None
        Crystal symmetries of the phase.
    color : str
        Name of phase color in Matplotlib's list of named colors.
    color_rgb : tuple
        RGB values of phase color, obtained from the color name.
    


No custom, private attributes.

Public attributes

In [60]:
[i for i in dir(Phase) if not i.startswith('_')]

['color', 'color_rgb', 'deepcopy', 'name', 'symmetry']

In [61]:
print(Phase.color.__doc__)


None


From PhaseList by...

... phase name

In [62]:
phases['austenite']

<name: austenite. symmetry: 432. color: tab:blue>

... phase ID

In [63]:
phases[1]

<name: austenite. symmetry: 432. color: tab:blue>

Initialize

In [64]:
Phase('au', symmetry='m-3m', color='salmon')

<name: au. symmetry: m-3m. color: salmon>

# <a id='examples'></a> 6. Examples

Plot phase map with property as alpha

In [65]:
cm.plot_phase('ci');

In [75]:
# Alternatively, return data array plotted, figure, image and axes
#data, fig, im, ax = cm.plot_phase(
#    overlay='ci',
#    scalebar=False,
#    legend=False,
#    padding=False,
#)

# And e.g. save it as map of pixels or use it as navigatior in HyperSpy
#plt.imsave(os.path.join(datadir, 'phase_ci.png'), arr=data)

In [67]:
# Per phase
cm['austenite'].plot_phase('ci');

Plot property

In [68]:
cm.plot_prop('iq');

In [76]:
#data, fig, im, ax = cm.plot_prop(
#    prop='iq',
#    scalebar=False,
#    colorbar=False,
#    padding=False,
#    cmap='viridis',
#)

In [70]:
# Per phase
cm['ferrite'].plot_prop('iq', cmap='viridis');

In [71]:
# 2D slicing
cm[20:40, 50: 100].plot_prop('ci');

In [77]:
# Conditional slicing
cm[cm.ci > 0.81].plot_prop('ci', cmap='winter');

# Chained conditional slicing
#cm[(cm.ci > 0.81) & (cm.phase_id == 1)].plot_prop('ci', cmap='winter');

Plot histogram of a property per phase

In [79]:
# Change colors (for fun)
cm.phases['ferrite'].color = (1, 0, 0)  # or 'r' or 'red'
cm.phases['austenite'].color = (0, 1, 0)  # or 'lime'

# Property of interest
this_prop = 'ci'

# Plot phase map again to see color changes
cm.plot_phase(this_prop)

# Declare lists for plotting
data = []
labels = []
colors = []

# Get property values, name and color per phase
for _, p in cm.phases_in_map:
    labels.append(p.name)
    colors.append(p.color)

    mask = cm[p.name].indexed
    data.append(cm[p.name].prop[this_prop][mask])
    # Alternatively, without the two above lines:
    #data.append(cm[p.name].ci.compressed())
    # Need compressed() because cm['some_masking'].ci returns a
    # masked array, which does not play nicely with matplotlib.hist.
    # This works because properties are made available as properties via
    # __getattr__

# Nice bar plot with property histogram per phase
fig, ax = plt.subplots()
ax.hist(
    data,
    bins=20,
    histtype='bar',
    density=True,
    label=labels,
    color=colors
)
ax.set_xlabel(this_prop)
ax.set_ylabel("Frequency")
ax.legend();